In [1]:
import os
import re

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


tf.random.set_seed(9999)

In [2]:
def get_chk_dir_path()->str:
    return os.path.abspath('../chk')    

def get_dat_dir_path()->str:
    return os.path.abspath('../dat')

def get_train_csv_path()->str:
    dat_dir = get_dat_dir_path()
    # print(dat_dir)
    return os.path.join(dat_dir, "train.csv")

def get_test_csv_path()->str:
    dat_dir = get_dat_dir_path()
    return os.path.join(dat_dir, "test_x.csv")

def get_sample_sub_path()->str:
    dat_dir = get_dat_dir_path()
    return os.path.join(dat_dir, "sample_submission.csv")


In [3]:
train = pd.read_csv(get_train_csv_path())
test = pd.read_csv(get_test_csv_path())
sample_submission = pd.read_csv(get_sample_sub_path())

In [4]:
# preprocesscing

In [5]:
def alpha_num(txt:str)->str:
    return re.sub(r"[^A-Za-z0-9 ]", "", txt)

train["text"] = train["text"].apply(alpha_num)

In [6]:
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


In [7]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

def remove_stopwords(text:str) -> str:
    final_txt = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_txt.append(i.strip())
    return " ".join(final_txt)

In [8]:
train["text"] = train["text"].apply(alpha_num).apply(remove_stopwords)
test["text"] = test["text"].apply(alpha_num).apply(remove_stopwords)

In [9]:
train

,index,text,author
0,0,almost choking much much wanted say strange ex...,3
1,1,sister asked suppose,2
2,2,engaged one day walked perusing Janes last let...,1
3,3,captain porch keeping carefully way treacherou...,4
4,4,mercy gentlemen odin flung hands Dont write an...,3
...,...,...,...
54874,54874,Mr Smith odin whispered hardly dared hope come,2
54875,54875,told plan captain us settled details accomplis...,4
54876,54876,sincere wellwisher friend sister LUCY odin,1
54877,54877,wanted lend money,3


In [10]:
# x_train = train["text"].values

In [11]:
# x_train

In [12]:
x_train = np.array([x for x in train["text"]])
x_test = np.array([x for x in test["text"]])
y_train = np.array([x for x in train["author"]])

In [13]:
# x_train

In [14]:
# Modeling

In [15]:
vocab_size = 20000
embedding_dim = 128
max_length = 500
padding_type = "post"

In [16]:
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

In [17]:
list(word_index)[:10]

['odin', 'not', 'said', 'no', 'one', 'mr', 'will', 'upon', 'now', 'man']

In [18]:
train_sequence = tokenizer.texts_to_sequences(x_train)
train_padded = pad_sequences(train_sequence, padding=padding_type, maxlen=max_length)

test_sequence = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequence, padding=padding_type, maxlen=max_length)

In [19]:
train_padded

array([[  141,  7259,    20, ...,     0,     0,     0],
       [  217,    58,   221, ...,     0,     0,     0],
       [  682,     5,    59, ...,     0,     0,     0],
       ...,
       [ 2407, 19738,   126, ...,     0,     0,     0],
       [  316,  3532,   164, ...,     0,     0,     0],
       [  203,     2,   811, ...,     0,     0,     0]])

In [20]:
############################################################################
## this model part is from 텐서플로2와 머신러닝으로 시작하는 자연어처리 04. 텍스트분류
############################################################################

model_name = "rnn_classifier_en"
BATCH_SIZE=128
NUM_EPOCHS=10
VALID_SPLIT=0.1
MAX_LEN=train_padded.shape[1]

kargs = {
    "model_name": model_name,
    "vocab_size": vocab_size,
    "embedding_size": 100,
    "dropout_rate": .2,
    "lstm_dimension": 100,
    "dense_dimension": 150,
    "output_dimension": 5
}

In [21]:
class RNNClassifier(tf.keras.Model):
    def __init__(self, **kargs):
        super(RNNClassifier, self).__init__(name=kargs["model_name"])
        self.embedding = tf.keras.layers.Embedding(input_dim=kargs["vocab_size"],
                                                   output_dim=kargs["embedding_size"])
        self.lstm_1_layer = tf.keras.layers.LSTM(kargs["lstm_dimension"], return_sequences=True)
        self.lstm_2_layer = tf.keras.layers.LSTM(kargs["lstm_dimension"])
        
        self.dropout = tf.keras.layers.Dropout(kargs["dropout_rate"])
        self.fc1 = tf.keras.layers.Dense(units=kargs["dense_dimension"], activation="relu")
        self.fc2 = tf.keras.layers.Dense(units=kargs["output_dimension"], activation="softmax")


    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = self.lstm_1_layer(x)
        x = self.lstm_2_layer(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [22]:
model = RNNClassifier(**kargs)


In [23]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
model.build(train_padded.shape)
print(model.summary())            

Model: "rnn_classifier_en"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  2000000   
_________________________________________________________________
lstm (LSTM)                  multiple                  80400     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  80400     
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  15150     
_________________________________________________________________
dense_1 (Dense)              multiple                  755       
Total params: 2,176,705
Trainable params: 2,176,705
Non-trainable params: 0
_______________________________________

In [24]:
#num_epochs = 20
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", 
                                                      min_delta=0.0001, 
                                                      patience=2)

checkpoint_path = os.path.join( get_chk_dir_path(), model_name, "weights.h5" )
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 monitor="val_accuracy", 
                                                 verbose=1, 
                                                 save_best_only=True, 
                                                 save_weights_only=True)

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir, exist_ok=True)

history = model.fit(train_padded, y_train, 
                    batch_size=BATCH_SIZE, 
                    epochs=NUM_EPOCHS, 
                    verbose=2, 
                    validation_split=VALID_SPLIT, 
                    callbacks=[earlystop_callback, cp_callback])

Epoch 1/10

Epoch 00001: val_accuracy improved from -inf to 0.25601, saving model to d:\github\daewonyoon\misc\2020\10_dacon_predict_novelist\chk\rnn_classifier_en\weights.h5
386/386 - 29s - loss: 1.5722 - accuracy: 0.2743 - val_loss: 1.5773 - val_accuracy: 0.2560
Epoch 2/10

Epoch 00002: val_accuracy did not improve from 0.25601
386/386 - 29s - loss: 1.5704 - accuracy: 0.2763 - val_loss: 1.5776 - val_accuracy: 0.2560
Epoch 3/10

Epoch 00003: val_accuracy did not improve from 0.25601
386/386 - 29s - loss: 1.5702 - accuracy: 0.2765 - val_loss: 1.5787 - val_accuracy: 0.2560


In [25]:
model.load_weights(checkpoint_path)

pred = model.predict(test_padded)

In [26]:
pred

array([[0.23569211, 0.13302971, 0.22039609, 0.26814133, 0.14274077],
       [0.23569208, 0.13302971, 0.22039609, 0.26814133, 0.14274077],
       [0.23569211, 0.13302971, 0.22039609, 0.26814133, 0.14274077],
       ...,
       [0.23569208, 0.13302971, 0.22039609, 0.26814133, 0.14274077],
       [0.23569211, 0.13302973, 0.22039609, 0.26814133, 0.14274077],
       [0.23569211, 0.13302973, 0.22039609, 0.26814133, 0.14274077]],
      dtype=float32)

In [27]:
pred.shape

(19617, 5)

In [28]:
test_padded.shape

(19617, 500)

In [29]:
sample_submission[[str(i) for i in range(5)]] = pred
sample_submission

,index,0,1,2,3,4
0,0,0.235692,0.13303,0.220396,0.268141,0.142741
1,1,0.235692,0.13303,0.220396,0.268141,0.142741
2,2,0.235692,0.13303,0.220396,0.268141,0.142741
3,3,0.235692,0.13303,0.220396,0.268141,0.142741
4,4,0.235692,0.13303,0.220396,0.268141,0.142741
...,...,...,...,...,...,...
19612,19612,0.235692,0.13303,0.220396,0.268141,0.142741
19613,19613,0.235692,0.13303,0.220396,0.268141,0.142741
19614,19614,0.235692,0.13303,0.220396,0.268141,0.142741
19615,19615,0.235692,0.13303,0.220396,0.268141,0.142741


In [30]:
sample_submission.to_csv("submission_rnn.csv", index=False, encoding="utf-8")